<a href="https://colab.research.google.com/github/kingluda/Geospatial-Analysis-and-Visualization/blob/main/Data_Preparation_for_Mapping_and_Random_Points_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Install required libraries
!pip install geopandas shapely fiona pyproj openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 85.7 MB/s eta 0:00:00


In [7]:
# ------------------ STEP 2: MOUNT GOOGLE DRIVE ------------------
from google.colab import drive
drive.mount('/content/drive')

# ------------------ STEP 3: IMPORT LIBRARIES ------------------
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import random
import zipfile
import os

# ------------------ STEP 4: UPLOAD FILES ------------------
from google.colab import files
uploaded = files.upload()  # Upload MMDA.zip and District_Opportunity_Counts.xlsx

# Unzip shapefile
with zipfile.ZipFile("MMDA.zip", 'r') as zip_ref:
    zip_ref.extractall("MMDA_shapefile")

# Detect .shp file
shp_files = [f for f in os.listdir("MMDA_shapefile") if f.endswith(".shp")]
if not shp_files:
    raise ValueError("No .shp file found.")
shp_path = os.path.join("MMDA_shapefile", shp_files[0])

# ------------------ STEP 5: READ AND PREPARE DATA ------------------
gdf = gpd.read_file(shp_path)
df_counts = pd.read_excel("District_Opportunity_Counts.xlsx")

# Clean column names
gdf.columns = gdf.columns.str.strip()
df_counts.columns = df_counts.columns.str.strip()

# Harmonize casing and spacing in district names
gdf['DISTRICT'] = gdf['DISTRICT'].str.upper().str.strip()
df_counts['District (MMDA)'] = df_counts['District (MMDA)'].str.upper().str.strip()

# Merge shapefile and Excel
merged = gdf.merge(df_counts, how='left', left_on='DISTRICT', right_on='District (MMDA)')

# ------------------ STEP 6: GENERATE RANDOM POINTS ------------------
def generate_random_points(polygon, num_points):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < num_points:
        pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(pnt):
            points.append(pnt)
    return points

points_list = []

for _, row in merged.iterrows():
    geom = row['geometry']
    district = row['DISTRICT']
    total_points = int(row['Total']) if pd.notnull(row['Total']) else 0

    if total_points > 0 and geom.is_valid and not geom.is_empty:
        points = generate_random_points(geom, total_points)
        for pt in points:
            points_list.append({
                "District": district,
                "geometry": pt
            })

# ------------------ STEP 7: EXPORT SHAPEFILE ------------------
if points_list:
    points_gdf = gpd.GeoDataFrame(points_list, geometry='geometry', crs=gdf.crs)

    output_dir = "/content/drive/MyDrive/Random_MMDA_Points"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, "random_points_by_total.shp")
    points_gdf.to_file(output_path)

    print("✅ Done! Shapefile saved to:", output_path)
else:
    print("⚠️ No points generated. Please check shapefile geometry and totals.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving MMDA.zip to MMDA (1).zip
✅ Done! Shapefile saved to: /content/drive/MyDrive/Random_MMDA_Points/random_points_by_total.shp
